## Analysis of Robinhood Portfolio

Project by: Pete Aguirre II

In this project, I will do a retuern:risk analysis on my current Robinhood stock portfolio with the help of multiple 
tools using:
- Python 3
- Jupyter Lab/Notebook
- Beautiful Soup
- Markowitz Efficent Frontier

In [ ]:
# Libraries Used 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
# import requests

import robin_stocks as r 
import pyotp
import json

from pandas_datareader import data as wb
from bs4 import BeautifulSoup as soup


# Magic
%matplotlib inline

## 1.) Data Collection (updated)

In [ ]:
# Robinhood Log In 
email = '###'
password = '###'

totp = pyotp.TOTP("My2factorAppHere").now()
log_in = r.login(email, password, expiresIn=500, by_sms=False, mfa_code=totp)

# Get stocks value
stonks = r.build_holdings()

# Logout 
r.logout()

In [ ]:
# Stonks Key : Value preview 
for key, value in stonks.items():
    print(key, value)

In [ ]:
# Company tickers
symbols = []
for k in  stonks.keys():
    symbols.append(k)
symbols = np.array(symbols)
symbols = pd.DataFrame(symbols, columns=['Symbol'])
symbols

In [236]:
# Other values
other_vals = pd.DataFrame.from_dict(stonks.values())
#other_vals = other_val.drop(['percent_change', 'equity_change', 'type', 'id'], axis=1)
other_vals
other_vals.rename(columns={'price':'Price', 'quantity':'Quantity', 'average_buy_price':'Average Price', 'equity':'Equity', 'name':'Name', 'pe_ratio':'P/E', 'percentage':'Percentage'})


#header_rename = ['price':Price', 'quantity':Quantity', 'average_price_buy':Average Price', 'equity':'Equity', 'name':'Name', 'pe_ratio':P/E', 'percentage':'Percentage']


#print(other_val.col())
#header_rename = ['Price', 'Quantity', 'Average Price', 'Equity', 'Name', 'P/E', 'Percentage']
#for i, j in range(len(other_val)-3):
#    other_val = other_val.rename(columns={other_val[i]: header_rename[j]})
#other_val.rename(columns={'price':'Price', 'quantity':'Quantity'})
#type(other_val.iloc[0])
#pd.DataFrame.rename(columns={'price':'Price', 'quantity':'Quantity', })
#other_val

,Price,Quantity,Average Price,Equity,percent_change,equity_change,type,Name,id,P/E,Percentage
0,3225.000000,1.00000000,2978.4800,3225.00,8.28,246.520000,stock,Amazon,c0bb3aec-bd1e-471e-a4f0-ca011cbec711,120.650000,24.87
1,455.267000,5.00000000,400.5540,2276.34,13.66,273.565000,stock,Apple,450dfc6d-5510-4d40-abfb-f633b7d9be3e,33.339900,17.55
2,172.000000,5.00000000,160.6400,860.00,7.07,56.800000,stock,Boeing,ae7f719c-ba1a-4207-8d94-af40fb7310f8,None,6.63
3,265.680000,10.00000000,246.0856,2656.80,7.96,195.944000,adr,Alibaba,b2e06903-5c44-46a4-bd42-2a696f9d68e1,32.151300,20.48
4,64.910000,8.00000000,64.1613,519.28,1.17,5.989600,adr,JD.com,b1e0ba21-bf54-454a-a409-89fe275bbe05,111.328000,4.00
5,43.640000,20.00000000,42.4100,872.80,2.90,24.600000,stock,Exxon Mobil,9133b38b-4917-4b5a-8eab-c029d60f9912,25.995700,6.73
6,33.030000,15.00000000,31.7173,495.45,4.14,19.690500,stock,Southwest Airlines,09bc1a2d-534d-49d4-add7-e0eb3be8e640,78.167800,3.82
7,27.040000,15.00000000,25.0280,405.60,8.04,30.180000,stock,Delta Air Lines,b9a6444e-ce3e-4186-be32-b82814d2b418,None,3.13
8,38.250000,10.00000000,36.9630,382.50,3.48,12.870000,stock,Pfizer,6ec6c70e-d686-4d73-b5a8-74fec96aca0e,15.264400,2.95
9,56.220000,10.00000000,56.5770,562.20,-0.63,-3.570000,adr,AstraZeneca,7aeae03d-a1ba-4670-af0e-b592378e7bfc,68.356900,4.33


In [ ]:
portfolio = pd.concat(symbols, other_val)
portfolio

In [ ]:
portfolio.info()

## 1.) Data Collection (Portfolio fr. Robinhood)

Obtain portfolio from Robinhood using BeautifulSoup4. 

In [ ]:
# On robinhood.com/account, begin copy and paste from html:
# <div class="_35i1NVinE8wTOLz3sUynSw"><header class="_1rWpCWWqvbg316kwNPlwYi">
# Note(s): I had to manipulate some html text in able to extract a span that
#          does not contain an attribute. I named it as: class="filler".
# Change:    <div class="_1bZB-iudENk38jTXhs7BIB"><span>
# Change to: <div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">
#       Another one was added for Total Return, called: class="filler2"
# Change:    </svg><span><span>
# Change to: </svg><span><span class="filler2">
html_doc = '''<div class="_35i1NVinE8wTOLz3sUynSw"><header class="_1rWpCWWqvbg316kwNPlwYi"><div class="_2YdFPnDqXSoDKmQtq28ML- _2Tpy7JyddSjzNzIEW6bQgP css-b8vnd1"><span class="css-h74qi0"><span class="css-13vshb9"><span class="css-1k2rdgi">Name</span></span></span><svg class="_1-Dd_YXn_flSIsvKPPGJF3 css-1bummxh" width="8" height="5" viewBox="0 0 8 5"><polygon fill-rule="evenodd" points="50 9 54 14 46 14" transform="rotate(-180 27 7)"></polygon></svg></div><div class="_2-4BkMtIykh6hAhu1CkOAi _2Tpy7JyddSjzNzIEW6bQgP css-b8vnd1"><span class="css-h74qi0"><span class="css-13vshb9"><span class="css-1k2rdgi">Symbol</span></span></span><svg class="_1-Dd_YXn_flSIsvKPPGJF3 css-1bummxh" width="8" height="5" viewBox="0 0 8 5"><polygon fill-rule="evenodd" points="50 9 54 14 46 14" transform="rotate(-180 27 7)"></polygon></svg></div><div class="_2Tpy7JyddSjzNzIEW6bQgP css-b8vnd1"><span class="css-h74qi0"><span class="css-13vshb9"><span class="css-1k2rdgi">Shares</span></span></span><svg class="_1-Dd_YXn_flSIsvKPPGJF3 css-1bummxh" width="8" height="5" viewBox="0 0 8 5"><polygon fill-rule="evenodd" points="50 9 54 14 46 14" transform="rotate(-180 27 7)"></polygon></svg></div><div class="_1aY3uEJAcFViGgVc3SRz4d _2Tpy7JyddSjzNzIEW6bQgP css-b8vnd1"><span class="css-h74qi0"><span class="css-13vshb9"><span class="css-1k2rdgi">Price</span></span></span><svg class="_1-Dd_YXn_flSIsvKPPGJF3 css-1bummxh" width="8" height="5" viewBox="0 0 8 5"><polygon fill-rule="evenodd" points="50 9 54 14 46 14" transform="rotate(-180 27 7)"></polygon></svg></div><div class="_2gJfY0FDaI4PWOsRbu1PPj _2Tpy7JyddSjzNzIEW6bQgP css-b8vnd1"><span class="css-h74qi0"><span class="css-13vshb9"><span class="css-1k2rdgi">Average Cost</span></span></span><svg class="_1-Dd_YXn_flSIsvKPPGJF3 css-1bummxh" width="8" height="5" viewBox="0 0 8 5"><polygon fill-rule="evenodd" points="50 9 54 14 46 14" transform="rotate(-180 27 7)"></polygon></svg></div><div class="Ue-PUFBPXUbpP5zhTrFKT _3QouEAcfZir2lV5xvc_C0b css-b8vnd1"><span class="css-zc0oik"><span class="css-13vshb9"><span class="css-1k2rdgi">Total Return</span></span></span><svg class="_1-Dd_YXn_flSIsvKPPGJF3 css-1n5qlcf" width="8" height="5" viewBox="0 0 8 5"><polygon fill-rule="evenodd" points="50 9 54 14 46 14" transform="rotate(-180 27 7)"></polygon></svg></div><div class="atrP1y1y_C9ULHV4BSwFj _2Tpy7JyddSjzNzIEW6bQgP css-b8vnd1"><span class="css-h74qi0"><span class="css-13vshb9"><span class="css-1k2rdgi">Equity</span></span></span><svg class="_1-Dd_YXn_flSIsvKPPGJF3 css-1bummxh" width="8" height="5" viewBox="0 0 8 5"><polygon fill-rule="evenodd" points="50 9 54 14 46 14" transform="rotate(-180 27 7)"></polygon></svg></div></header><div><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB" rel="" href="/stocks/AMZN"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">Amazon</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">AMZN</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">1</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$3,193.89</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$2,978.48</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$215.41</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$3,193.89</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/AAPL"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">Apple</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">AAPL</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">5</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$439.45</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$400.55</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$194.48</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$2,197.25</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/BABA"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">Alibaba</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">BABA</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">10</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$265.00</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$246.09</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$189.14</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$2,650.00</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/BA"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">Boeing</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">BA</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">5</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$175.31</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$160.64</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$73.35</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$876.55</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/XOM"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">Exxon Mobil</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">XOM</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">20</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$43.94</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$42.41</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$30.60</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$878.80</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/DAL"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">Delta Air Lines</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">DAL</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">15</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$27.00</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$25.03</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$29.58</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$405.00</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/LUV"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">Southwest Airlines</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">LUV</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">15</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$33.62</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$31.72</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$28.54</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$504.30</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/PFE"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">Pfizer</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">PFE</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">10</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$39.00</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$36.96</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$20.37</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$390.00</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/JD"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">JD.com</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">JD</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">5</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$65.52</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$64.17</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$6.75</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$327.60</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/AGNC"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">AGNC Investment</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">AGNC</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">15</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$13.67</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$13.65</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$0.33</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$205.05</span></div></div></div></div></a><a class="rh-hyperlink qD5a4psv-CV7GnWdHxLvn n7JlFw29Ep8AXGLvITeC4 _3DXBkJzx8C8DnFG0wXGZ50 _2LZkydeTTkR9XsU9ETIiTB _2nPaH1Do76n14Fi4_DN3Ae" rel="" href="/stocks/AZN"><div class="_3sbUrrmoSkcTz1WFZ1cZ85"><div class="_2pf2FyleeFNQa8GgYJPzME"><div class="_37g6R7bjpn_GE9wrF8xPab"><div class="_1bZB-iudENk38jTXhs7BIB"><div class="_2YdFPnDqXSoDKmQtq28ML-"><span class="css-13vshb9"><span class="_2jKxrvkjD73sLQEfH5NTgT">AstraZeneca</span></span></div></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2-4BkMtIykh6hAhu1CkOAi">AZN</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="filler">10</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_1aY3uEJAcFViGgVc3SRz4d">$56.12</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="_2gJfY0FDaI4PWOsRbu1PPj">$56.58</span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="Ue-PUFBPXUbpP5zhTrFKT"><svg class="_1gFu7pxJKo2ybEddsSwwbq _2mbK0Mw-5HM4q7eeuDqOau" width="9" height="8" viewBox="0 0 9 8"><polygon fill-rule="evenodd" points="4 0 8 7 0 7"></polygon></svg><span><span class="filler2">$4.57</span></span></span></div><div class="_1bZB-iudENk38jTXhs7BIB"><span class="atrP1y1y_C9ULHV4BSwFj">$561.20</span></div></div></div></div></a></div></div>'''

In [ ]:
soup = soup(html_doc, 'html.parser')
print(soup.prettify()[0:50000])

In [ ]:
# Extracting headers
headers_txt = []
headers = soup.find_all('span', {'class':'css-1k2rdgi'})
for i in range(len(headers)):
    headers_txt.append(headers[i].text)
    
my_portfolio = pd.DataFrame(columns=[headers_txt])
my_portfolio

In [ ]:
# Extracting company names
names_txt = []
names = soup.find_all('span', {'class':'_2jKxrvkjD73sLQEfH5NTgT'})

for i in range(len(names)):
    names_txt.append(names[i].text)
names_txt = np.array(names_txt)
#names_txt

In [ ]:
# Extracting company tickers
ticks_txt = []
ticks = soup.find_all('span', {'class':'_2-4BkMtIykh6hAhu1CkOAi'})

for i in range(len(ticks)):
    ticks_txt.append(ticks[i].text)
ticks_txt = np.array(ticks_txt)
#ticks_txt

In [ ]:
# Extracting company shares
shares_txt = []
shares = soup.find_all('span', {'class':'filler'})

for i in range(len(shares)):
    shares_txt.append(int(float(shares[i].text)))
shares_txt = np.array(shares_txt)
#shares_txt

In [ ]:
# Extracting company prices
prices_txt = []
prices = soup.find_all('span', {'class':'_1aY3uEJAcFViGgVc3SRz4d'})

for i in range(len(prices)):
    prices_txt.append(prices[i].text)
prices_txt = np.array(prices_txt)
#prices_txt

In [ ]:
# Extracting average price obtained
avg_txt = []
avg = soup.find_all('span', {'class':'_2gJfY0FDaI4PWOsRbu1PPj'})

for i in range(len(avg)):
    avg_txt.append(avg[i].text)
avg_txt = np.array(avg_txt)
#avg_txt

In [ ]:
# Extracting total return
returns_txt = []
returns = soup.find_all('span', {'class':'filler2'})

for i in range(len(returns)):
    returns_txt.append(returns[i].text)
returns_txt = np.array(returns_txt)
#returns_txt

In [ ]:
# Extracting equity
equity_txt = []
equity = soup.find_all('span', {'class':'atrP1y1y_C9ULHV4BSwFj'})

for i in range(len(ticks)):
    equity_txt.append(equity[i].text)
equity_txt = np.array(equity_txt)
#equity_txt

In [ ]:
my_portfolio = pd.DataFrame({'Name':names_txt, 'Symbol':ticks_txt, 'Shares':shares_txt, 'Price':prices_txt, 'Average Cost':avg_txt, 'Total Return':returns_txt, 'Equity':equity_txt})
my_portfolio

In [ ]:
my_portfolio.info()

## 2.) Data Cleaning

In [ ]:
# Take out all dollar signs in data
# Note: Running this a second time will create an error since the 
#       values would have already been converted to floats
dollar_sign = ['Price', 'Average Cost', 'Total Return', 'Equity']
for d in dollar_sign:
    my_portfolio[d] = my_portfolio[d].str.replace(",", "")
    my_portfolio[d] = my_portfolio[d].str.replace("$", "")
    my_portfolio[d] = pd.to_numeric(my_portfolio[d])

In [ ]:
# Confirm that values needed has been converted to floats
my_portfolio.info()

In [ ]:
my_portfolio

In [ ]:
# Convert any '.' to '-' in symbols
#sym = my_portfolio['Symbol']
tick_new = my_portfolio['Symbol'].str.replace(".", "-")

In [ ]:
# Confirm that '.' has been replaced by '-' on symbol
my_portfolio

In [ ]:
# EXTRA DATA GATHERING HERE

total_sum = (my_portfolio['Average Cost']*my_portfolio['Shares']).sum()
weights = []
for i in range(len(my_portfolio)):
    weights.append(round((my_portfolio['Average Cost'][i]*my_portfolio['Shares'][i])/total_sum, 2))
    
my_portfolio = pd.DataFrame({'Name':names_txt, 'Symbol':tick_new, 'Shares':shares_txt, 'Price':prices_txt, 'Average Cost':avg_txt, 'Total Return':returns_txt, 'Equity':equity_txt, 'Weight':weights})
my_portfolio

In [ ]:
my_portfolio['Weight'].sum()
my_portfolio.to_csv('portfolio.csv')

## For manually obtained CSV

In [ ]:
my_portfolio = pd.read_csv('hypothetical_portfolio.csv')
my_portfolio

In [ ]:
#dollar_sign = ['Average Cost', 'Total Return', 'Equity']
#for d in dollar_sign:
#    my_portfolio[d] = pd.to_numeric(my_portfolio[d])
my_portfolio.info()

In [ ]:
# Only run to download weights
total_sum = (my_portfolio['Average Cost']*my_portfolio['Shares']).sum()
weights = []
for i in range(len(my_portfolio)):
    weights.append(round((my_portfolio['Average Cost'][i]*my_portfolio['Shares'][i])/total_sum, 2))
    
#my_portfolio = pd.DataFrame({'Name':names_txt, 'Symbol':tick_new, 'Shares':shares_txt, 'Average Cost':avg_txt, 'Total Return':returns_txt, 'Equity':equity_txt, 'Weight':weights})
weights = pd.DataFrame({'Weights': weights})
weights.to_csv('weights.csv', index=False)


## 3.) Data Collection (Historical Prices)

In [ ]:
# Collect historical prices
tickers = my_portfolio['Symbol']
start_date = '2000-01-01'
my_data = pd.DataFrame()
for t in tickers:
    my_data[t] = wb.DataReader(t, data_source='yahoo', start=start_date)['Adj Close']

In [ ]:
my_data = my_data.dropna()
my_data.info()

In [ ]:
sec_returns = np.log(my_data/my_data.shift(1))
sec_returns

sec_returns.info()

In [ ]:
sec_returns.head()

In [ ]:
sec_returns.tail()

## Data Analysis

In [ ]:
sec_returns

In [ ]:
for t in tickers:

    
    print("")
    print("#",t)
    print("DAILY")
    print("Daily Return:", round(sec_returns[t].mean()*100, 4), "%")
    print("Daily Risk:", round(sec_returns[t].std()*100, 4), "%")

    print("")

    print("ANNUAL")
    mean_return = round((sec_returns[t].mean()*250)*100, 4)
    std_return = round((sec_returns[t].std()*250**.5)*100, 4)
    print("Annual Return:", mean_return, "%")
    print("Annual Risk:", std_return, "%")
    

    
    

In [ ]:
sec_returns.idxmin() 

In [ ]:
sec_returns.idxmax()

In [ ]:
# Covariance 
# Determines if there is a relationship between two stocks, whether they move together:
# positively, negatively, or neutral 
return_cov = sec_returns.cov()
return_cov

In [ ]:
# Covariance 
# Determines if there is a relationship between two stocks, whether they move together:
# positively, negatively, or neutral 
return_cov = sec_returns.cov()*250
return_cov

In [ ]:
# Correlations
# Determines how close the relationship are between two stocks
return_corr = sec_returns.corr()
return_corr

In [ ]:
# Calculating Diversifiable and Non-Diversifiable

# Portfolio variance
pflio_var = np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight']))
print("Portfolio Variance", pflio_var)

# Portfolio volatility 
#pflio_vol = (np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight'])))**.5
#print("Portfolio Volatility:", pflio_vol)

# or...

pflio_vol2 = np.sqrt(pflio_var)
print("Portfolio Volatility:", pflio_vol2)


# Calculating variance annually for each tickers
var_a = []
for t in tickers:
    var_a.append(sec_returns[t].var()*250)

var_a = np.array(var_a)
variances = pd.DataFrame({'Name':names_txt, 'Variance':var_a})
print("")
# Diversifiable Risk
# div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0] - ...
#                  ... - my_portfolio['Weight'][n]**2*variances['Variance'][n]
div_risk = 0   
for i in range(len(my_portfolio)):
    if i==0:
        div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0]
    else:
        div_risk -= my_portfolio['Weight'][i]**2*variances['Variance'][i]
print("Diversifiable Risk:", div_risk)

print("")
# Non Diversifiable Risk
non_div_risk1 = pflio_var - div_risk
print("Non-Diversifiable Risk:", non_div_risk1)

print("")
non_div_risk2 = 0
for i in range(len(my_portfolio)):
    non_div_risk2 += my_portfolio['Weight'][i]**2*variances['Variance'][i]
    #print(my_portfolio['Name'][i], non_div_risk2)

print("")
print("Non-Diversifiable Risk:", non_div_risk2)
print("Non-Diversifiable Risk:", non_div_risk2 == non_div_risk1)


##test = my_portfolio['Weight'].diff()
#print(test)

In [ ]:
# FOR THE MANUAL MODE
# Calculating Diversifiable and Non-Diversifiable

# Portfolio variance
pflio_var = np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight']))
print("Portfolio Variance", pflio_var)

# Portfolio volatility 
pflio_vol = (np.dot(my_portfolio['Weight'].T, np.dot(sec_returns.cov()*250, my_portfolio['Weight'])))**.5
print("Portfolio Volatility:", pflio_vol)

# Calculating variance annually for each tickers
var_a = []
for t in tickers:
    var_a.append(sec_returns[t].var()*250)

var_a = np.array(var_a)
variances = pd.DataFrame({'Name':my_portfolio['Name'], 'Variance':var_a})

# Diversifiable Risk
# div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0] - ...
#                  ... - my_portfolio['Weight'][n]**2*variances['Variance'][n]
div_risk = 0   
div_risk = pflio_var - my_portfolio['Weight'][0]**2*variances['Variance'][0]
for i in range(len(my_portfolio)-1):
    div_risk -= my_portfolio['Weight'][i+1]**2*variances['Variance'][i+1]
    #print(my_portfolio['Name'][i], div_risk)
div_risk = div_risk - my_portfolio['Weight'][len(my_portfolio)-1]**2*variances['Variance'][len(my_portfolio)-1]
print("Diversifiable Risk:", div_risk)

# Non Diversifiable Risk
non_div_risk1 = pflio_var - div_risk
print("Non-Diversifiable Risk:", non_div_risk1)

non_div_risk2 = 0
for i in range(len(my_portfolio)):
    non_div_risk2 += my_portfolio['Weight'][i]**2*variances['Variance'][i]
print("Non-Diversifiable Risk:", non_div_risk2)
print("Non-Diversifiable Risk:", non_div_risk2 == non_div_risk1)


##test = my_portfolio['Weight'].diff()
#print(test)

In [ ]:
no_assets = len(tickers)
no_assets

In [ ]:
pflio_ret = []
pflio_vol = [] 
pflio_wei = []

for x in range(10000):
    weights = np.random.random(no_assets)
    weights /= np.sum(weights)
    
    pflio_wei.append(weights)
    pflio_ret.append(np.sum(weights*sec_returns.mean())*250)
    pflio_vol.append(np.sqrt(np.dot(weights.T, np.dot(sec_returns.cov()*250, weights))))
    #print(x, weights)

pflio_wei = np.array(pflio_wei)
pflio_ret = np.array(pflio_ret)
pflio_vol = np.array(pflio_vol)

#print(np.sum(weights))
#pflio_returns, pflio_volatilities

In [ ]:
pflio_scenarios = pd.DataFrame({'Return': pflio_ret, 'Volatility': pflio_vol})
pflio_scenarios = pflio_scenarios.sort_values('Return', ascending=True)

In [ ]:
pflio_scenarios.head()
#pflio_scenarios['Return']

In [ ]:
pflio_scenarios.tail(3500)

In [ ]:
# Efficient Frontier 
pflio_scenarios.plot(x='Volatility', y='Return', kind='scatter', figsize=(10,6));
plt.xlabel('Expected Volatility')
plt.ylabel('Expected Return')
eff_front = plt.savefig("efficient_frontier2.png")

In [ ]:
# For Testing purposes
pfolio_wei2 = []
tickers2 = []
print(pflio_wei[19169])
tickers2 = np.array(tickers)
print(tickers2)
print(np.sum(pflio_wei[19169]))
pflio_wei2 = pflio_wei[19169]


#ideal_portfolio = pd.DataFrame(columns=tickers2)
#ideal_portfolio = pd.DataFrame({'Symbol': tickers2, 'Weights': pflio_wei2})
ideal_portfolio = pd.DataFrame({'Weights': pflio_wei2})
ideal_portfolio.to_csv('ideal_weights.csv', index=False)

## Data Visualization

In [ ]:
(my_data/my_data.iloc[0]*100).plot(figsize=(20,15))

In [ ]:
sns.set_style('whitegrid')
regression = sns.pairplot(sec_returns[1:], kind="reg")
regression = regression.savefig("regression.png")
regression

In [ ]:
plt.subplots(figsize=(15, 15))
heatmap = sns.heatmap(return_corr, annot=True, square=True, cmap='coolwarm')
heatmap.savefig("heatmap.png")
heatmap
#plt.show()